# Data preparation

In [1]:
# Separe aqui seus dados pra fazer a inferencia

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os
import csv
from pathlib import Path
import json
import pandas as pd
from tqdm.auto import tqdm
from ast import literal_eval
from tqdm import tqdm

# caminho_base = '/content/drive/MyDrive/red_pill_analises/classificador/oficial_comentarios_classificacao/'
caminho_base = ''
caminho_folds = os.path.join(caminho_base, 'folds/')

# caminho_embeddings = os.path.join(caminho_base, 'videos/embeddings_minilm/')
# caminho_resultados = os.path.join(caminho_base, 'videos/resultados_minilm/')

caminho_embeddings = os.path.join(caminho_base, 'comentarios/embeddings_qwen_4/')
caminho_resultados = os.path.join(caminho_base, 'comentarios/resultados_qwen_4/')

# Preciso dos seguintes arquivos: Os 10 arquivos de folds, csv de vizinhos e

# Setup Llama 3.1:8b Instruct

In [2]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='llm_inferences.log',
    filemode='a'
)

In [3]:
local_path = "/home/jovyan/datalake/models/llama3-8b"

print(f"Loading tokenizer from {local_path}...")
tokenizer = AutoTokenizer.from_pretrained(
    local_path,
    local_files_only=True,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("pad_token defined as eos_token for tokenizer.")

print(f"Loading model from {local_path}...")

model = AutoModelForCausalLM.from_pretrained(
    local_path,
    local_files_only=True,
    trust_remote_code=True,
    dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="cuda"
)

Loading tokenizer from /home/jovyan/datalake/models/llama3-8b...
pad_token defined as eos_token for tokenizer.
Loading model from /home/jovyan/datalake/models/llama3-8b...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda


In [5]:
generation_params = {
    "max_new_tokens": 256,
    "do_sample": False,
    #"temperature": 0.7,
    "truncation": False,
    "return_full_text": False,
}
batch_size = 64

In [6]:
def save_batch_to_csv(batch, results_to_save):
    """
    Save a batch result into a CSV file, creating the header if the file
    does not exist or is empty.

    Args:
        batch (int): Batch ID
        results_to_save (list): List of llm results obtained (prompt + output)
    """

    output_csv_file = f"llm_inferences/llama3.1-8b-instruct/{batch}.csv"
    fieldnames = ["original_id", "original_prompt", "llm_response"]

    output_dir = Path(output_csv_file).parent
    output_dir.mkdir(parents=True, exist_ok=True)

    write_header = not os.path.exists(output_csv_file) or os.path.getsize(output_csv_file) == 0

    try:
        with open(output_csv_file, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            if write_header:
                writer.writeheader()
                logging.info(f"Header written to '{output_csv_file}'.")

            writer.writerows(results_to_save)

    except IOError as e:
        logging.error(f"Error on saving batch to CSV '{output_csv_file}': {e}")

In [7]:
PROGRESS_FILE = 'inferences.json'

def load_progress():
    """Load the progress from a JSON file."""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            logging.info(f"Progress file '{PROGRESS_FILE}' found. Loading state.")
            return json.load(f)
    logging.info(f"No progress file '{PROGRESS_FILE}' found. Starting from zero.")
    return {}

def save_progress(progress_data):
    """Save the current progress into a JSON file"""
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress_data, f, indent=4)

In [8]:
progress = load_progress()

# Prompts

In [9]:
all_tests = [
    # --- Teste 1: Zero-Shot, Português, Sem Conceito ---
    {
        "name": "teste1_pt_sem_conceito",
        "system_prompt_base": (
            "Você é um classificador de texto altamente preciso de comentários do Youtube. "
            "Sua única função é analisar o comentário do usuário e responder estritamente com '0' ou '1'.\n"
            "A pergunta que você deve responder é: O texto é {task_definition}?"
            "1: O texto É {task_definition}\n"
            "0: O texto NÃO é {task_definition}\n"
            "Não forneça nenhuma explicação, apenas o número."
        ),
        "task_definitions": {
            "toxicidade": "TÓXICO",
            "misoginia": "MISÓGINO"
        },
        "few_shot_config": {
            "toxicidade": {"output_prefix": "results_toxic"},
            "misoginia": {"output_prefix": "results_mysogyny"}
        }
    },

    # --- Teste 2: Zero-Shot, Inglês, Sem Conceito ---
    {
        "name": "teste2_en_sem_conceito",
        "system_prompt_base": (
            "You are a highly accurate text classifier for Youtube comments. "
            "Your sole function is to analyze the user's comment and respond strictly with '0' or '1'.\n"
            "The question you must answer is: Is the text {task_definition}?\n"
            "1: The text IS {task_definition}\n"
            "0: The text is NOT {task_definition}\n"
            "Do not provide any explanation, just the number."
        ),
        "task_definitions": {
            "toxicidade": "TOXIC",
            "misoginia": "MISOGYNISTIC"
        },
        "few_shot_config": {
            "toxicidade": {"output_prefix": "results_toxic"},
            "misoginia": {"output_prefix": "results_mysogyny"}
        }
    },

    # --- Teste 3: Zero-Shot, Português, Com Conceito ---
    {
        "name": "teste3_pt_com_conceito",
        "system_prompt_base": (
            "Você é um classificador de texto altamente preciso de comentários do Youtube. "
            "Sua única função é analisar o comentário do usuário e responder estritamente com '0' ou '1'.\n"
            "Você deve analisar se o texto contém {task_definition}. Assim, responda:"
            "1: Se o texto se encaixa em qualquer uma das categorias de conceito listadas \n"
            "0: Se o texto NÃO se encaixa em qualquer uma das categorias de conceito listadas\n"
            "Não forneça nenhuma explicação, apenas o número."
        ),
        "task_definitions": {
            "toxicidade": "TOXICIDADE - Utilize as seguintes definições para guiar sua classificação: \n1. Ódio e Assédio:\n1.1 Doxing: Divulgação de informações pessoais de um alvo para o público em geral, com o objetivo de prejudicá-lo.\n1.2 Ataque à identidade: Consiste em ataques severos a indivíduos ou grupos com base em sua afiliação a um grupo protegido ou vulnerável.\n1.3 Falsa representação de identidade: É a disseminação de declarações que comunicam deturpações pejorativas, estereótipos e outras generalizações insultuosas sobre populações protegidas ou vulneráveis.\n1.4 Insulto: Ataque pessoal direcionado, como xingamentos, comentários negativos, depreciativos e inflamatórios sobre o indivíduo.\n1.5 Agressão Sexual: Inclui avanços sexuais indesejados, sexualização indesejável e compartilhamento não consensual de conteúdo sexual, entre outras formas de conversas sexuais não solicitadas.\n1.6 Ameaça de Violência: Refere-se a conteúdo que expressa o desejo de matar ou infligir dano físico, celebra atos violentos ou incita outros a cometerem violência.\n2. Dano autoinfligido:\n2.1 Promoção de transtorno alimentar: Abrange conteúdo que promove transtornos alimentares como escolhas de estilo de vida legítimas ou glorifica corpos excessivamente magros.\n2.2 Autolesão: Ideação de se machucar fisicamente de propósito para lidar com o sofrimento emocional, incluindo também a ideação suicida.\n3. Dano ideológico:\n3.1 Extremismo, terrorismo e crime organizado: Inclui conteúdo relacionado ao recrutamento, elogio ou assistência a grupos extremistas, terroristas ou de crime organizado.\n3.2 Desinformação: É a informação falsa ou enganosa, que pode ser disseminada com ou sem a intenção deliberada de prejudicar.\n4. Exploração:\n4.1 Serviços sexuais adultos: Refere-se à promoção ou solicitação de serviços sexuais ilegais que exploram participantes frequentemente vulneráveis.\n4.2 Material de abuso sexual infantil: É definido como conteúdo que envolve o abuso e a exploração sexual de qualquer pessoa menor de dezoito anos.\n4.3 Golpes: São tentativas de enganar uma pessoa para que ela forneça fundos ou informações sensíveis usando técnicas fraudulentas ou invasivas.\n",
            "misoginia": "MISOGINIA - Utilize as seguintes definições para guiar sua classificação: \n1. Ameaças, planos para prejudicar e incitamento:\n1.1 Ameaças de danos: Expressar desejo de prejudicar física, sexual ou emocionalmente uma mulher ou grupo de mulheres.\n1.2 Incitamento e incentivo ao dano: Encorajar ou justificar que outros prejudiquem uma mulher ou grupo de mulheres.\n2. Depreciação:\n2.1 Ataques descritivos: Fazer generalizações negativas sobre as habilidades, aparência, intelecto ou caráter das mulheres.\n2.2 Ataques agressivos e emotivos: Expressar ódio ou nojo contra as mulheres, usando insultos de gênero ou acusações infundadas.\n2.3 Ataques desumanizantes e objetificação sexual explícita: Comparar mulheres a animais/objetos ou reduzi-las a objetos sexuais.\n3. Animosidade:\n3.1 Uso casual de gírias de gênero, profanidades e insultos: Usar termos pejorativos para mulheres de forma casual, não como um ataque direto.\n3.2 Diferenças imutáveis de gênero e estereótipos de gênero: Afirmar diferenças 'naturais' entre homens e mulheres para implicar inferioridade ou limitar papéis (inclui a maioria das piadas sexistas).\n3.3 Elogios de gênero ambíguos: Elogiar mulheres de forma que reforce estereótipos de fragilidade, ou que reduza seu valor à sua atratividade.\n3.4 Explicações condescendentes ou conselhos indesejados: Oferecer conselhos paternalistas a mulheres sobre assuntos que elas dominam ('mansplaining').\n4. Discussão Preconceituosa:\n4.1 Apoio ao mau tratamento de mulheres individuais: Justificar, negar ou subestimar o mau tratamento de uma mulher.\n4.2 Apoio à discriminação sistêmica contra mulheres como grupo: Justificar, negar ou subestimar a discriminação contra as mulheres como um grupo.\n"
        },
        "few_shot_config": {
            "toxicidade": {"output_prefix": "results_toxic"},
            "misoginia": {"output_prefix": "results_mysogyny"}
        }
    },

    # --- Teste 4: Zero-Shot, Inglês, Com Conceito ---
    {
        "name": "teste4_en_com_conceito",
        "system_prompt_base": (
            "You are a highly accurate text classifier for Youtube comments. "
            "Your sole function is to analyze the user's comment and respond strictly with '0' or '1'.\n"
            "You must analyze if the text contains {task_definition}. Accordingly, respond:\n"
            "1: If the text fits into any of the listed concept categories\n"
            "0: If the text does NOT fit into any of the listed concept categories\n"
            "Do not provide any explanation, just the number."
        ),
        "task_definitions": {
            "toxicidade": "TOXICITY - Use the following definitions to guide your classification: \n1. Hate and Harassment:\n1.1 Doxing: Disclosure of a target's personal information to the general public with the intent to harm them.\n1.2 Identity Attack: Consists of severe attacks on individuals or groups based on their affiliation with a protected or vulnerable group.\n1.3 Misrepresentation of Identity: Is the dissemination of statements that communicate pejorative misrepresentations, stereotypes, and other insulting generalizations about protected or vulnerable populations.\n1.4 Insult: A directed personal attack, such as name-calling, negative, derogatory, and inflammatory comments about the individual.\n1.5 Sexual Aggression: Includes unwanted sexual advances, unwelcome sexualization, and non-consensual sharing of sexual content, among other forms of unsolicited sexual conversations.\n1.6 Threat of Violence: Refers to content that expresses the desire to kill or inflict physical harm, celebrates violent acts, or incites others to commit violence.\n2. Self-Harm:\n2.1 Promotion of Eating Disorders: Covers content that promotes eating disorders as legitimate lifestyle choices or glorifies excessively thin bodies.\n2.2 Self-Injury: Ideation of purposely physically hurting oneself to cope with emotional distress, including suicidal ideation.\n3. Ideological Harm:\n3.1 Extremism, Terrorism, and Organized Crime: Includes content related to the recruitment, praise, or assistance of extremist, terrorist, or organized crime groups.\n3.2 Disinformation: Is false or misleading information, which can be disseminated with or without the deliberate intention to cause harm.\n4. Exploitation:\n4.1 Adult Sexual Services: Refers to the promotion or solicitation of illegal sexual services that exploit often-vulnerable participants.\n4.2 Child Sexual Abuse Material: Is defined as content that involves the sexual abuse and exploitation of any person under the age of eighteen.\n4.3 Scams: Are attempts to deceive a person into providing funds or sensitive information using fraudulent or invasive techniques.\n",
            "misoginia": "MISOGYNY - Use the following definitions to guide your classification: \n1. Threats, plans to harm, and incitement:\n1.1 Threats of harm: Expressing a desire to physically, sexually, or emotionally harm a woman or group of women.\n1.2 Incitement and encouragement of harm: Encouraging or justifying others to harm a woman or group of women.\n2. Derogation:\n2.1 Descriptive attacks: Making negative generalizations about women's skills, appearance, intellect, or character.\n2.2 Aggressive and emotive attacks: Expressing hatred or disgust against women, using gendered slurs or baseless accusations.\n2.3 Dehumanizing attacks and explicit sexual objectification: Comparing women to animals/objects or reducing them to sexual objects.\n3. Animosity:\n3.1 Casual use of gendered slurs, profanity, and insults: Using derogatory terms for women casually, not as a direct attack.\n3.2 Immutable gender differences and gender stereotypes: Asserting 'natural' differences between men and women to imply inferiority or limit roles (includes most sexist jokes).\n3.3 Ambiguous gendered praise: Praising women in a way that reinforces stereotypes of fragility, or that reduces their value to their attractiveness.\n3.4 Condescending explanations or unsolicited advice: Offering paternalistic advice to women on subjects they are knowledgeable about ('mansplaining').\n4. Prejudiced Discussion:\n4.1 Support for the mistreatment of individual women: Justifying, denying, or understating the mistreatment of a woman.\n4.2 Support for systemic discrimination against women as a group: Justifying, denying, or understating discrimination against women as a group.\n"
        },
        "few_shot_config": {
            "toxicidade": {"output_prefix": "results_toxic"},
            "misoginia": {"output_prefix": "results_mysogyny"}
        }
    }
]


# Código: Few Shot COM EMBEDDINGS



In [10]:
# Para rodar para todos de 1 só vez
import pandas as pd
import logging
from tqdm import tqdm
import transformers
import os

# --- Configurações Iniciais ---
# Tamanho do lote para a pipeline do LLM (quantos prompts enviar para a GPU de uma vez)
LLM_BATCH_SIZE = 64 #
transformers.logging.set_verbosity_error()
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filename='processamento_llm_dynamic.log', filemode='w')

# =========================================================
# PARTE 1: CARREGAMENTO MESTRE DE TODOS OS COMENTÁRIOS
# =========================================================
# Vamos carregar todos os 10 folds em memória UMA VEZ para
# criar um "banco de dados" (lookup) de id_comentario -> texto.
# Isso é MUITO mais rápido do que ler arquivos dentro do loop.

logging.info("Carregando todos os 10 dataframes de folds para lookup...")
master_lookups = {} # Dicionário para guardar os DFs mestres

try:
    for task_key in ['toxicidade', 'misoginia']:
        sufixo = "_TOXICIDADE" if task_key == 'toxicidade' else "_MISOGINIA"
        label_col = 'final_toxicidade' if task_key == 'toxicidade' else 'final_misoginia'

        all_dfs = []
        for i in range(1, 6):
            # Assumindo que os arquivos estão na mesma pasta, como você mencionou
            fname = f"comentarios_fold_{i}_estratificado{sufixo}.csv"
            all_dfs.append(pd.read_csv(os.path.join(caminho_folds, fname)))

        # Concatena os 5 folds da tarefa em um único DataFrame
        df_master = pd.concat(all_dfs)

        # Define o ID como índice para buscas O(1) (super rápidas)
        # E seleciona apenas as colunas que importam
        master_lookups[task_key] = df_master[['id_comentario_anonimizado', 'comentario', label_col]].set_index('id_comentario_anonimizado')

        logging.info(f"Lookup mestre para '{task_key}' criado com {len(master_lookups[task_key])} vídeos.")

except FileNotFoundError as e:
    logging.error(f"Erro fatal: Não foi possível carregar os arquivos de fold. Verifique se todos os 10 CSVs (videos_fold_...csv) estão na pasta. Arquivo faltando: {e.filename}")
    raise e

logging.info("Carregamento mestre de todos os folds concluído.")


# =========================================================
# PARTE 2: LOOP DE PROCESSAMENTO DINÂMICO
# =========================================================

for test_config in all_tests:
    test_name = test_config["name"]
    SYSTEM_PROMPT_BASE = test_config["system_prompt_base"]
    TASK_DEFINITIONS = test_config["task_definitions"]
    FEW_SHOT_CONFIG = test_config["few_shot_config"]

    logging.info(f"############################################################")
    logging.info(f"##### INICIANDO GRUPO DE TESTE: {test_name} #####")
    logging.info(f"############################################################")

    for fold_number in range(1, 6):
        for task_name_pt, config in FEW_SHOT_CONFIG.items():

            task_key = 'misoginia' if task_name_pt == 'misoginia' else 'toxicidade'
            label_column = 'final_misoginia' if task_key == 'misoginia' else 'final_toxicidade'
            sufixo_arquivo = "_MISOGINIA" if task_key == 'misoginia' else "_TOXICIDADE"

            logging.info(f"============================================================")
            logging.info(f"PROCESSANDO: Teste '{test_name}' - Fold {fold_number} - Tarefa: {task_key}")
            logging.info(f"============================================================")

            # --- 1. Definir arquivos de entrada para este loop ---
            # Arquivo com a lista de comentários a processar (o fold de teste)
            test_list_filename = f"comentarios_fold_{fold_number}_estratificado{sufixo_arquivo}.csv"
            # Arquivo com os vizinhos para CADA comentário de teste
            neighbor_filename = f"nearest_neighbors_{task_key}_fold_{fold_number}.csv" # (sem data)

            try:
                # Carrega a lista de comentários que vamos classificar
                df_test_list = pd.read_csv(os.path.join(caminho_folds, test_list_filename))
                # Carrega o DF de vizinhos e o indexa por ID para busca rápida
                df_neighbors = pd.read_csv(os.path.join(caminho_embeddings, neighbor_filename)).set_index('id_comentario_anonimizado')
            except FileNotFoundError as e:
                logging.warning(f"Arquivo não encontrado: {e.filename}. Pulando Fold {fold_number} / Tarefa {task_key}.")
                continue

            # Pega o lookup mestre de textos/rótulos que já carregamos
            df_master_lookup = master_lookups[task_key]

            # --- 2. Preparar Prompt ---
            task_definition = TASK_DEFINITIONS.get(task_key, "")
            dynamic_system_prompt = SYSTEM_PROMPT_BASE.format(task_definition=task_definition)
            prefix = "Comment:" if 'en' in test_name else "Comentário:"

            # --- 3. Loop de Processamento (em Lotes) ---
            all_results_for_run = []

            # Itera sobre o DataFrame de TESTE em lotes (para a pipeline do LLM)
            for start in tqdm(range(0, len(df_test_list), LLM_BATCH_SIZE), desc=f"{test_name} - F{fold_number} ({task_key})"):
                end = start + LLM_BATCH_SIZE
                chunk_df = df_test_list.iloc[start:end]

                batch_conversations = [] # Lista de prompts (conversas) para o lote
                batch_original_data = [] # Lista para guardar IDs e textos originais

                # --- 3a. Construir os prompts dinâmicos para o lote ---
                for _, row in chunk_df.iterrows():
                    comment_id = row['id_comentario_anonimizado']
                    comment_text = row['comentario']

                    try:
                        # Pega a linha de vizinhos para este comentário específico
                        neighbor_ids = df_neighbors.loc[comment_id]
                    except KeyError:
                        logging.warning(f"ID {comment_id} não encontrado no arquivo de vizinhos {neighbor_filename}. Pulando este comentário.")
                        continue

                    # Inicia a conversa com o prompt do sistema
                    conversation = [{"role": "system", "content": dynamic_system_prompt}]

                    # Constrói os exemplos (2 neg, 3 pos)
                    examples_to_add = []
                    try:
                        # 2 Exemplos Negativos (Rótulo 0)
                        for i in range(1, 3): # neg_1_id, neg_2_id
                            neg_id = neighbor_ids[f'neg_{i}_id']
                            neg_text = df_master_lookup.loc[neg_id]['comentario']
                            examples_to_add.append({"role": "user", "content": f"{prefix} '{neg_text}'"})
                            examples_to_add.append({"role": "assistant", "content": "0"})

                        # 3 Exemplos Positivos (Rótulo 1)
                        for i in range(1, 4): # pos_1_id, pos_2_id, pos_3_id
                            pos_id = neighbor_ids[f'pos_{i}_id']
                            pos_text = df_master_lookup.loc[pos_id]['comentario']
                            examples_to_add.append({"role": "user", "content": f"{prefix} '{pos_text}'"})
                            examples_to_add.append({"role": "assistant", "content": "1"})

                    except KeyError as e:
                        logging.error(f"Erro de lookup de ID: {e}. O vizinho do video {comment_id} não foi encontrado no lookup mestre. Pulando este comentário.")
                        continue

                    # Adiciona os 5 exemplos de few-shot
                    conversation.extend(examples_to_add)

                    # Adiciona o prompt final do usuário (o comentário a ser classificado)
                    formatted_msg = f"{prefix} '{comment_text}'"
                    conversation.append({"role": "user", "content": formatted_msg})

                    # Adiciona a conversa completa ao lote
                    batch_conversations.append(conversation)
                    # Guarda os dados originais para salvar depois
                    batch_original_data.append({
                        "id": comment_id,
                        "text": comment_text
                    })

                if not batch_conversations:
                    continue

                # --- 3b. Executar a pipeline do LLM no lote ---
                batch_prompts_formatted = tokenizer.apply_chat_template(
                    batch_conversations, tokenize=False, add_generation_prompt=True
                )
                batch_results = pipe(batch_prompts_formatted, **generation_params)

                # --- 3c. Coletar resultados do lote ---
                results_to_save_this_chunk = []
                for i, result in enumerate(batch_results):
                    llm_response = result[0]['generated_text'].strip()
                    original_data = batch_original_data[i]

                    results_to_save_this_chunk.append({
                        "original_id": original_data["id"],
                        "original_prompt": original_data["text"],
                        "llm_response": llm_response
                    })

                if results_to_save_this_chunk:
                    all_results_for_run.extend(results_to_save_this_chunk)

            # --- 4. Salvar resultados do Fold ---
            if all_results_for_run:
                output_prefix = config["output_prefix"]
                # Nome do arquivo final é dinâmico, baseado nos vizinhos
                output_filename = f"{output_prefix}_{test_name}_fold_{fold_number}_DYNAMIC_NEIGHBORS.csv"

                final_df = pd.DataFrame(all_results_for_run)
                final_df.to_csv(os.path.join(caminho_resultados, output_filename), index=False)
                logging.info(f"Resultados (Dinâmicos) salvos em: {output_filename}")
            else:
                logging.warning(f"Nenhum resultado gerado para Teste '{test_name}', Fold {fold_number}, Tarefa: {task_key}")

logging.info("============================================================")
logging.info("TODOS OS TESTES DINÂMICOS FORAM CONCLUÍDOS!")
logging.info("============================================================")


teste4_en_com_conceito - F5 (misoginia): 100%|██████████| 7/7 [01:06<00:00,  9.53s/it]


In [11]:
"oi"

'oi'